In [209]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
yrs=[1983,1984,1985]
arr=[]
links=[]
for yr in yrs:
    html=requests.get("https://www.profootballarchives.com/"+str(yr)+"usfl.html")
    soup=BeautifulSoup(html.text)
    stats=soup.find("div",{"id":"stats"})
    table=stats.find_all("table")[0]
    for a in table.find_all("a"):
        links.append(a.get("href"))
        url="https://www.profootballarchives.com/"+str(a.get("href"))
        html=requests.get(url)
        soup=BeautifulSoup(html.text)
        
        link=soup.find("title")
        link=str(link.text)
        team=link.split(str(yr))
        team=team[1]
        team=team.replace(" (USFL)","")
        team=team.replace(" - Pro Football Archives","")
        table=soup.find_all("table")[4]
        
        for tr in table.find_all("tr"):
            h=[]
            for td in tr.find_all("td"):
                h.append(td.text)
                
            h.append(team)
            h.append(yr)
            arr.append(h)
yrs=[2022]
links=[]
html=requests.get("https://www.profootballarchives.com/2022usflbir.html")
soup=BeautifulSoup(html.text)
table=soup.find_all("table")[3]
for a in table.find_all("a"):
    if '<a></a>' not in str(a):
        links.append(a.get("href"))
for link in links:
    for yr in yrs:
        
        link=str(link)
        if '83' in link:
            link=link.split("83")
            link=link[1]
        if '84' in link:
            link=link.split("84")
            link=link[1]
        if '85' in link:
            link=link.split("85")
            link=link[1]
        link=link
        url="https://www.profootballarchives.com/"+link
        html=requests.get(url)
        soup=BeautifulSoup(html.text)
        
        link=soup.find("title")
        link=str(link.text)
        team=link.split(str(yr))
        team=team[1]
        team=team.replace(" (USFL)","")
        team=team.replace(" - Pro Football Archives","")
        table=soup.find_all("table")[3]
        
        for tr in table.find_all("tr"):
            h=[]
            for td in tr.find_all("td"):
                h.append(td.text)
                
            h.append(team)
            h.append(2022)
            arr.append(h)
df=pd.concat([pd.DataFrame(arr)])
df.columns=['Day','Date','VenueType','team2','score','result','OT','Location','Venue','Attendance','team1','season']
df=df.astype({'VenueType':'string'})

df=df[df.Date.notna()]
df=df[df.score.notna()]
df['score2']=0
df['score1']=0
for row in df.itertuples():
    score=str(row.score)
    score=score.split("-")
    s1=score[0]
    s2=score[1]
    df.at[row.Index,'score1']=s1
    df.at[row.Index,'score2']=s2
df['Date']=df['Date'].str.replace("/","")
df=df.astype({"team1":"string","team2":"string",'score2':float,'score1':float,"Date":"string"})  
df['team1']=df['team1'].str.replace(" ","")
df['team2']=df['team2'].str.replace(" ","")
for row in df.itertuples():
    f=df[(df.Date==row.Date)&(df.team2==row.team1)&(df.team1==row.team2)]
    if len(f)>0:
        df=df.drop(row.Index)


In [242]:
import math
import numpy as np
df=pd.read_csv("https://github.com/easella/usfl/raw/main/usflscores.csv")
df=df.astype({"team1":"string","team2":"string",'score2':float,'score1':float,'Date':"string"})
df['team1']=df['team1'].str.replace(" ","")
df['team2']=df['team2'].str.replace(" ","")
for row in df.itertuples():
    f=df[(df.Date==row.Date)&(df.team2==row.team1)&(df.team1==row.team2)]
    if len(f)>0:
        df=df.drop(row.Index)
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
homef=65
w=0

df.sort_values(by='Date',inplace=True)
l=0
class Elo:
  global winnerHome
  def __init__(self,k,g=1,homefield = 182.5):
    self.ratingDict   = {}  
    self.k        = k
    self.g        = g
  def addPlayer(self,name,rating = 1500):
    self.ratingDict[name] = rating
  def gameOver(self, winner, loser, winnerHome,neutral,wins,loses):
    homef=50
    wb=0
    lb=0
    if winnerHome and neutral!='N':
      result = self.expectResult(self.ratingDict[winner] + homef, self.ratingDict[loser])
    if winnerHome==False and neutral!='N':
      result = self.expectResult(self.ratingDict[winner], self.ratingDict[loser] + homef)
    if neutral=='N':
      result = self.expectResult(self.ratingDict[winner], self.ratingDict[loser])
    wins=float(wins)
    loses=float(loses)
    if wins==loses:
        tie=True
    else:
        tie=False
    if tie:
        mult=(math.log(0+1))*(2.2/1*0.001+2.2)
        shift=(self.k*mult)*(0.5 - result)  
        self.ratingDict[winner] +=shift
        self.ratingDict[loser]  -=shift
    if tie !=True:
        win=eloLeague.ratingDict[winner]
        lose=eloLeague.ratingDict[loser]
        if neutral==True:
          win=(eloLeague.ratingDict[winner])
          lose=(eloLeague.ratingDict[loser])
        if winnerHome and neutral!=True:
          win=(homef+eloLeague.ratingDict[winner])
          lose=(eloLeague.ratingDict[loser])
        if winnerHome!=True and neutral!=True:
          lose=(homef+eloLeague.ratingDict[loser])
          win=(eloLeague.ratingDict[winner])
        global zy
        if(win>lose):
          zy=(win-lose)
        if(win==lose):
          zy=(win-lose)
        if(win<lose):
          zy=(win-lose)
        mult=(math.log((wins-loses)+1))*(2.2/(zy)*0.001+2.2)
        shift=(self.k*mult)*(1 - result)  
        self.ratingDict[winner] +=shift
        self.ratingDict[loser]  -=shift
  def expectResult(self, p1, p2):
        global exp
        exp = (p2-p1)/400
        n2=10
        j=exp
        o=1/((n2**(j))+1)
        return float(o)
teams=set(df.team1.tolist()+df.team2.tolist())
eloLeague=Elo(k=20)
currSeason=1983
for team in teams:
    eloLeague.addPlayer(team,rating=1500)
for row in df.itertuples():
    if float(row.season)>currSeason:
        season=float(row.season)
        lsdf=df[df.season<season]
        ls=set(lsdf.team1.tolist()+lsdf.team2.tolist())
        csdf=df[df.season==season]
        cs=set(csdf.team1.tolist()+csdf.team2.tolist())
        
                
       
            
       
        
        
        for key in eloLeague.ratingDict.keys():
            
            if(key  in ls and key in cs):
                eloLeague.ratingDict[key]=(eloLeague.ratingDict[key]*(2/3))+(1500*(1/3))
                
        currSeason=float(row.season)
    if row.score1>row.score2 or row.score1==row.score2:
        if row.VenueType=='A':
            hfa=0
            if row.VenueType!='N':
                hfa=65
            if (eloLeague.ratingDict[row.team1])>(eloLeague.ratingDict[row.team2]+hfa):
                w+=1
            else:
                l+=1
            winnerhome=False
        else:
            hfa=0
            if row.VenueType!='N':
                hfa=65
            if (eloLeague.ratingDict[row.team1]+hfa)>(eloLeague.ratingDict[row.team2]):
                w+=1
            if (eloLeague.ratingDict[row.team1]+hfa)<(eloLeague.ratingDict[row.team2]):
                l+=1
            winnerhome=True
        eloLeague.gameOver(row.team1,row.team2,winnerhome,row.VenueType,row.score1,row.score2)
    if row.score1<row.score2:
        if row.VenueType=='A':
            hfa=0
            if row.VenueType!='N':
                hfa=65
            if (eloLeague.ratingDict[row.team1])>(eloLeague.ratingDict[row.team2]+hfa):
                w+=1
            if (eloLeague.ratingDict[row.team1])<(eloLeague.ratingDict[row.team2]+hfa):
                l+=1
            winnerhome=True
        else:
            winnerhome=False
        eloLeague.gameOver(row.team2,row.team1,winnerhome,row.VenueType,row.score2,row.score1)

In [237]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df

,Unnamed: 0,Day,Date,VenueType,team2,score,result,OT,Location,Venue,Attendance,team1,season,score2,score1
0,782,Sat,1985-02-23,A,TampaBayBandits,7-35,L,NaN,"Tampa, FL",Tampa Stadium,45095.0,OrlandoRenegades,1985.0,35.0,7.0
1,828,Sun,1985-02-24,A,OaklandInvaders,10-31,L,NaN,"Oakland, CA",Oakland-Alameda County Coliseum,23622.0,DenverGold,1985.0,31.0,10.0
2,932,Sun,1985-02-24,H,HoustonGamblers,33-34,L,NaN,"Los Angeles, CA",Memorial Coliseum,18828.0,LosAngelesExpress,1985.0,34.0,33.0
3,892,Sun,1985-02-24,A,ArizonaOutlaws,7-9,L,NaN,"Tempe, AZ",Sun Devil Stadium,20351.0,PortlandBreakers,1985.0,9.0,7.0
4,668,Sun,1985-02-24,A,BirminghamStallions,28-38,L,NaN,"Birmingham, AL",Legion Field,34785.0,NewJerseyGenerals,1985.0,38.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473,823,Sat,1985-07-06,A,MemphisShowboats,28-19,W,NaN,"Memphis, TN",Liberty Bowl,37796.0,OaklandInvaders,1985.0,19.0,28.0
474,599,Sat,1984-07-07,H,LosAngelesExpress,35-25,W,NaN,"Tempe, AZ",Sun Devil Stadium,33188.0,ArizonaWranglers,1984.0,25.0,35.0
475,735,Sun,1985-07-07,A,BirminghamStallions,28-14,W,NaN,"Birmingham, AL",Legion Field,23250.0,BaltimoreStars,1985.0,14.0,28.0
476,364,Sun,1984-07-08,A,PhiladelphiaStars,10-20,L,NaN,"Philadelphia, PA",Franklin Field,26616.0,BirminghamStallions,1984.0,20.0,10.0


In [243]:
df

,Unnamed: 0,Day,Date,VenueType,team2,score,result,OT,Location,Venue,Attendance,team1,season,score2,score1
107,110,Sun,1983-03-06,A,WashingtonFederals,28-7,W,NaN,"Washington, DC",RFK Stadium,38010.0,ChicagoBlitz,1983.0,7.0,28.0
105,194,Sun,1983-03-06,H,NewJerseyGenerals,20-15,W,NaN,"Los Angeles, CA",Memorial Coliseum,34002.0,LosAngelesExpress,1983.0,15.0,20.0
108,214,Sun,1983-03-06,H,PhiladelphiaStars,7-13,L,NaN,"Denver, CO",Mile High Stadium,45102.0,DenverGold,1983.0,13.0,7.0
109,132,Sun,1983-03-06,H,BostonBreakers,21-17,W,NaN,"Tampa, FL",Tampa Stadium,42437.0,TampaBayBandits,1983.0,17.0,21.0
106,234,Sun,1983-03-06,H,OaklandInvaders,0-24,L,NaN,"Tempe, AZ",Sun Devil Stadium,45167.0,ArizonaWranglers,1983.0,24.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,1107,Sat,2022-06-25,A,NewJerseyGenerals,19-14,W,NaN,"Canton, OH",Tom Benson Hall of Fame Stadium,NaN,PhiladelphiaStars,2022.0,14.0,19.0
418,1055,Sat,2022-06-25,A,BirminghamStallions,17-31,L,NaN,"Canton, OH",Tom Benson Hall of Fame Stadium,NaN,NewOrleansBreakers,2022.0,31.0,17.0
419,1093,Sat,2022-06-25,A,BirminghamStallions,17-31,L,NaN,"Canton, OH",Tom Benson Hall of Fame Stadium,NaN,NewOrleansBreakers,2022.0,31.0,17.0
472,1017,Sun,2022-07-03,H,BirminghamStallions,33-30,W,NaN,"Canton, OH",Tom Benson Hall of Fame Stadium,NaN,PhiladelphiaStars,2022.0,30.0,33.0


In [222]:
hfa=[]
n=df[df.VenueType!='N']
for row in n.itertuples():
    if row.VenueType=='A':
        hfa.append(row.score2-row.score1)
    else:
        hfa.append(row.score1-row.score2)
    

In [223]:
np.mean(hfa)

2.5991561181434597

In [245]:
print(100/(w+l)*w)

33.711048158640224


In [218]:
df.to_csv("usflscores.csv")

In [229]:
l2s={k: v for k, v in sorted(eloLeague.ratingDict.items(), key=lambda item: item[1],reverse=True)}
for key in l2s.keys():
    print(key,eloLeague.ratingDict[key])

PhiladelphiaStars 1900.1638128352813
BaltimoreStars 1745.2955200937088
MichiganPanthers 1713.2562837758528
BostonBreakers 1642.9069999272517
BirminghamStallions 1616.4908969121088
TampaBayBandits 1574.4760242409377
HoustonGamblers 1564.4790551176432
OaklandInvaders 1550.4285006949744
ArizonaWranglers 1549.5607056491328
ChicagoBlitz 1526.240621009748
MemphisShowboats 1509.402532768997
ArizonaOutlaws 1495.3554230811692
NewJerseyGenerals 1468.9607045907394
JacksonvilleBulls 1451.2748433106258
SanAntonioGunslingers 1431.2671106495982
LosAngelesExpress 1399.891472520459
OrlandoRenegades 1387.6910566720674
DenverGold 1384.891745890725
NewOrleansBreakers 1360.2009108232235
PortlandBreakers 1337.5259444867816
WashingtonFederals 1297.726952013573
OklahomaOutlaws 1205.622344292275
PittsburghMaulers 1102.870503115878
